
Using dbutils.fs.ls command to check the file that been uploaded

In [0]:
#Using dbutils.fs.ls command to check the file that been uploaded

dbutils.fs.ls("/FileStore/tables/")

Out[110]: [FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1706721501000),
 FileInfo(path='dbfs:/FileStore/tables/accounts2/', name='accounts2/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1706718610000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023.zip', name='clinicaltrial_2023.zip', size=52785750, modificationTime=1709406838000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023___test/', name='clinicaltrial_2023___test/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_

In [0]:
dbutils.fs.cp("/FileStore/tables/clinicaltrial_2023___test.zip", "file:/tmp/")

Out[111]: True


Reusable code to unzip all files

In [0]:
fileroot = "clinicaltrial_2023___test"

dbutils.fs.cp("FileStore/tables/" + fileroot + ".zip", "file:/tmp/")

import os
os.environ['fileroot'] = fileroot


In [0]:
%sh

unzip -d /tmp /tmp/$fileroot.zip

Archive:  /tmp/clinicaltrial_2023___test.zip
  inflating: /tmp/clinicaltrial_2023.csv  


In [0]:
dbutils.fs.mkdirs("FileStore/tables/" + fileroot)

Out[114]: True

In [0]:
dbutils.fs.mv("file:/tmp/clinicaltrial_2023.csv", "/FileStore/tables/" + fileroot, True)

Out[115]: True

In [0]:
dbutils.fs.head("/FileStore/tables/clinicaltrial_2023___test/clinicaltrial_2023.csv")

[Truncated to first 65536 bytes]
Out[116]: 'NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date\r\n"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019\r\n"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING 

In [0]:
rdd1 = sc.textFile("/FileStore/tables/clinicaltrial_2023___test/clinicaltrial_2023.csv")

rdd1.take(3)

Out[117]: ['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date',
 '"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019',
 '"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING \\t Cerebrovascular Function|C

In [0]:
#Extract header

header = rdd1.first()

print(header)

NCT Number \t Study Title \t Acronym \t Study Status \t Conditions \t Interventions \t Sponsor \t Collaborators \t Enrollment \t Funder Type \t Study Type \t Study Design,Start Date,Completion Date


Data Cleansing

In [0]:
#Let's split Header

# Extract header
header = rdd1.first()

# Split the header based on \t
header_elements = header.split("\\t")

# Split the last element based on comma
last_element_split = header_elements[-1].split(',')

# Combine the split elements into the header list
header_elements[-1:] = last_element_split

# Print the modified header
print(header_elements)

['NCT Number ', ' Study Title ', ' Acronym ', ' Study Status ', ' Conditions ', ' Interventions ', ' Sponsor ', ' Collaborators ', ' Enrollment ', ' Funder Type ', ' Study Type ', ' Study Design', 'Start Date', 'Completion Date']


In [0]:
#Define function to split lines in text

def split_line(line):
    # Split the line by tab character
    parts = line.split("\\t")
    # Split the last part by commas and extend the list
    last_part_split = parts[-1].split(",")
    return parts[:-1] + last_part_split

# Apply the split_line function to the input list
rdd2 = rdd1.map(split_line)

rdd2.take(2)

Out[120]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design',
  'Start Date',
  'Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR',
  ' OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT"',
  '20 August 2018',
  '28 February 2019']]

In [0]:
#Extract first line data
first_line = rdd2.map(lambda x: x[-1])

first_line.take(4)

Out[121]: ['Completion Date', '28 February 2019', '2024-08', '2014-12']

In [0]:
#Check the first row

Header_row = rdd2.take(1)

print(Header_row)

[['NCT Number ', ' Study Title ', ' Acronym ', ' Study Status ', ' Conditions ', ' Interventions ', ' Sponsor ', ' Collaborators ', ' Enrollment ', ' Funder Type ', ' Study Type ', ' Study Design', 'Start Date', 'Completion Date']]


In [0]:
# Extract Start Date and Completion Date

rdd3 = rdd2.map(lambda row: (row[-2], row[-1]))

rdd3.collect()  #show all data in rdd

Out[123]: [('Start Date', 'Completion Date'),
 ('20 August 2018', '28 February 2019'),
 ('25 October 2023', '2024-08'),
 ('2013-08', '2014-12'),
 ('01 October 2020', '01 October 2023'),
 ('01 July 2010', '30 October 2019'),
 ('05 May 2017', '09 October 2020'),
 ('01 September 2022', '15 June 2023'),
 ('01 February 2024', '01 March 2026'),
 ('11 May 2016', '13 December 2017'),
 ('2016-02', '2017-05'),
 ('13 July 2017', '30 June 2026'),
 ('2004-01', '2005-01'),
 ('09 February 2023', '17 November 2024'),
 ('2015-03', '2016-12'),
 ('13 June 2013', '02 November 2021'),
 ('13 June 2018', '24 December 2018'),
 ('2009-01', '2009-12'),
 ('2009-08', '2011-07'),
 ('2009-09', '2010-03'),
 ('02 December 2021', '31 December 2023'),
 ('2003-03', '2004-04'),
 ('30 May 2023', '30 May 2025'),
 ('15 July 2019', '01 September 2020'),
 ('15 April 2020', '31 May 2020'),
 ('2014-09', '2019-02'),
 ('01 July 2023', '01 December 2024'),
 ('01 January 2018', '31 December 2024'),
 ('06 May 2022', '06 November 202

In [0]:
#Create date without the Header

#create Date Header

Date_Header = rdd3.first()

print(Date_Header)

#Extract date without Header

rdd4 = rdd3.map(lambda x: x[:2])

('Start Date', 'Completion Date')


In [0]:
#Clean the date and convert to a consistent date format

from datetime import datetime

# Define a function to clean date data
def clean_date(date_str):
    if not date_str:
        return ""
        
    formats = ['%d %B %Y', '%Y %m %d', '%d %m %Y', '%Y-%m']
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt).strftime('%d %B %Y')
        except ValueError:
            pass
    return date_str

# Clean the date data in the RDD
cleaned_dates = rdd3.map(lambda x: (clean_date(x[0]), clean_date(x[1])))

# Collect and print the cleaned date data
for date_pair in cleaned_dates.collect():
    print(date_pair)

('Start Date', 'Completion Date')
('20 August 2018', '28 February 2019')
('25 October 2023', '01 August 2024')
('01 August 2013', '01 December 2014')
('01 October 2020', '01 October 2023')
('01 July 2010', '30 October 2019')
('05 May 2017', '09 October 2020')
('01 September 2022', '15 June 2023')
('01 February 2024', '01 March 2026')
('11 May 2016', '13 December 2017')
('01 February 2016', '01 May 2017')
('13 July 2017', '30 June 2026')
('01 January 2004', '01 January 2005')
('09 February 2023', '17 November 2024')
('01 March 2015', '01 December 2016')
('13 June 2013', '02 November 2021')
('13 June 2018', '24 December 2018')
('01 January 2009', '01 December 2009')
('01 August 2009', '01 July 2011')
('01 September 2009', '01 March 2010')
('02 December 2021', '31 December 2023')
('01 March 2003', '01 April 2004')
('30 May 2023', '30 May 2025')
('15 July 2019', '01 September 2020')
('15 April 2020', '31 May 2020')
('01 September 2014', '01 February 2019')
('01 July 2023', '01 December 202

In [0]:
## Create an rdd excluding the 'Start" and 'Completion" date

rdd5 = rdd2.map(lambda x: x[:12])

rdd5.take(1)

Out[126]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design']]

In [0]:
#Zip RDDs together and flatten the rdd

# Zip the RDDs together column-wise
zipped_rdd = rdd5.zip(cleaned_dates)

# Add start and completion date columns to each tuple in the zipped RDD as a tuple
flattened_rdd = zipped_rdd.map(lambda x: x[0] + list(x[1]))

# Output the result
flattened_rdd.take(2)

Out[129]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design',
  'Start Date',
  'Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR',
  '20 August 2018',
  '28 February 2019']]

Exploratory data analysis

In [0]:
#Check the count of values by column header

# Extract the header row
header_row = flattened_rdd.first()

# Count the total number of non-empty values for each column
column_counts = flattened_rdd.flatMap(lambda row: zip(header_row, row)) \
                   .filter(lambda x: x[1] != '') \
                   .map(lambda x: (x[0], 1)) \
                   .reduceByKey(lambda a, b: a + b) \
                   .collect()

# Output the column counts
for column, count in column_counts:
    print(f"Column '{column}': {count}")

Column 'NCT Number ': 478698
Column ' Interventions ': 477801
Column ' Sponsor ': 477801
Column ' Collaborators ': 477775
Column ' Study Design': 467588
Column ' Study Title ': 478698
Column ' Study Status ': 478698
Column ' Funder Type ': 475705
Column ' Study Type ': 472627
Column 'Completion Date': 438184
Column ' Acronym ': 478697
Column ' Conditions ': 477815
Column ' Enrollment ': 477049
Column 'Start Date': 471145


In [0]:
#Likewise, Check missing values by column headers

# Extract the header row
header_row = flattened_rdd.first()

# Count the total number of empty values for each column
empty_counts = flattened_rdd.flatMap(lambda row: zip(header_row, row)) \
                  .filter(lambda x: x[1] == '') \
                  .map(lambda x: (x[0], 1)) \
                  .reduceByKey(lambda a, b: a + b) \
                  .collect()

# Output the column counts
for column, count in empty_counts:
    print(f"Column '{column}': {count}")

Column ' Study Design': 10187
Column ' Interventions ': 882
Column ' Sponsor ': 882
Column ' Collaborators ': 908
Column ' Study Type ': 5174
Column 'Completion Date': 12204
Column ' Funder Type ': 2096
Column 'Start Date': 5182
Column ' Enrollment ': 752
Column ' Conditions ': 883
Column ' Acronym ': 1


In [0]:
##Number of Distinct Studies

# Extract the header row
header_row = flattened_rdd.first()

# Find the index of the 'NCT Number' column
nct_number_index = header_row.index('NCT Number ')

# Count the number of distinct studies
distinct_study_count = flattened_rdd.map(lambda row: row[nct_number_index]) \
                          .distinct() \
                          .count()

# Output the number of distinct studies
print("Number of distinct studies:", distinct_study_count)

Number of distinct studies: 478684
